### Usage example of `pygmo2.zdt` problem wrapper in currently defined Search Space

[pygmo2](https://github.com/esa/pagmo2) contain build-in definition of ZDT problems (*see Zitzler–Deb–Thiele's function 1..6 [here](https://en.wikipedia.org/wiki/Test_functions_for_optimization#Test_functions_for_multi-objective_optimization)*)

These functions provide a use-case to practice the multi-objective optimization.
Created `PgZdtWrapper` class is a wrapper for `pygmo2`'s ZDT implementation.
It exposes possibility to instantiate the problems in terms of our Search Space, sample the problem solutions (Configurations), evaluating them on-the-fly.

Note, since because of the restriction:
only the categorical hyperparameter could have a children, the Search Space still have a root of categorical hyperparameter with single category problem name - a useless bunch of information.

In [2]:
%cd ..

/media/sem/B54BE5B22C0D3FA81/TUD/Master/code/experiments


In [12]:
import pygmo as pg
import pandas as pd
from typing import List, MutableMapping

from core_entities.search_space import NominalHyperparameter, FloatHyperparameter, Hyperparameter

class PgZdtWrapper:
    """
    Wrapper for the synthetic ZDT (Zitzler–Deb–Thiele's function) multi-variate, multiobjective problem from Pygmo framework.
    Initizalization parameters:
        - problem_id: integer between 1 and 6. Defines problem type, more here: https://esa.github.io/pagmo2/docs/cpp/problems/zdt.html
        - number_of_dimensions: integer, the range for each problem type is specific.
    """
    def __init__(self, problem_id: int, number_of_dimensions: int):
        self.problem = pg.problem(pg.zdt(prob_id=problem_id, param=number_of_dimensions))

        root = NominalHyperparameter("problem", [self.problem.get_name()])
        for x, bounds in enumerate(zip(*self.problem.get_bounds())):
            child = FloatHyperparameter(name=f"x_{x}", lower=bounds[0], upper=bounds[1])
            root.add_child_hyperparameter(child)

        self.search_space = root

    def sample(self, n: int) -> List[MutableMapping]:
        """
            Sample n new data points in the initialized ZDT problem.
        """
        sampled = []
        for _ in range(n):
            config = {}
            while not self.search_space.validate(config, recursive=True):
                self.search_space.generate(config)
            params = list(filter(lambda x: isinstance(x, (int, float)), config.values()))
            results = self.problem.fitness(params)
            config.update({f"f_{k}": v for k, v in enumerate(results)})
            sampled.append(config)
        return sampled
    
    def evaluate(self, config: MutableMapping) -> MutableMapping:
        params = list(filter(lambda x: isinstance(x, (int, float)), config.values()))
        results = self.problem.fitness(params)
        config.update({f"f_{k}": v for k, v in enumerate(results)})
        return config
        

In [13]:
zdt = PgZdtWrapper(problem_id = 4, number_of_dimensions = 4)
print(zdt.problem)
print(zdt.search_space)

Problem name: ZDT4
	Global dimension:			4
	Integer dimension:			0
	Fitness dimension:			2
	Number of objectives:			2
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [0, -5, -5, -5]
	Upper bounds: [1, 5, 5, 5]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: basic

NominalHyperparameter 'problem'.
├ Default category: 'ZDT4'.
├ Categories:
├  ZDT4:
├+   FloatHyperparameter 'x_0'
├    | Lower boundary: 0.0, upper boundary: 1.0.
├    | Default value: 0.5.
├+   FloatHyperparameter 'x_1'
├    | Lower boundary: -5.0, upper boundary: 5.0.
├    | Default value: 5.0.
├+   FloatHyperparameter 'x_2'
├    | Lower boundary: -5.0, upper boundary: 5.0.
├    | Default value: 5.0.
├+   FloatHyperparameter 'x_3'
├    | Lower boundary: -5.0, upper boundary: 5.0.
├    | Default value: 5.0.


#### Sampling the Configurations from the problem

In [17]:
configs = zdt.sample(n=20)
configs_as_df = pd.DataFrame(data=configs)
print(configs_as_df)

   problem       x_0       x_1       x_2       x_3       f_0        f_1
0     ZDT4  0.481465 -3.939946 -2.400175  1.808933  0.481465  47.511338
1     ZDT4  0.387463 -3.542724  0.921715 -4.533488  0.387463  37.674068
2     ZDT4  0.828848 -3.106117  4.402333 -3.940919  0.828848  55.276900
3     ZDT4  0.921614  2.756472  1.120845  4.605785  0.921614  60.198033
4     ZDT4  0.011982 -1.509626  0.944054 -4.128968  0.011982  33.521448
5     ZDT4  0.395476 -2.462325 -4.638446 -0.002136  0.395476  37.318946
6     ZDT4  0.409745  2.536187  3.602344  1.459451  0.409745  28.395075
7     ZDT4  0.236934 -1.462970 -0.454250 -4.523088  0.236934  24.369457
8     ZDT4  0.744856  3.364925 -2.845770 -4.023890  0.744856  55.122943
9     ZDT4  0.185915  1.164276 -3.371652  3.831493  0.185915  65.185600
10    ZDT4  0.000328  0.301844 -2.395622  3.465492  0.000328  45.032904
11    ZDT4  0.373332 -2.591948 -4.557424 -3.449864  0.373332  46.413780
12    ZDT4  0.577537  2.040174 -3.814422 -4.503374  0.577537  52